# Notebook 3: Implementación de la Estrategia Momentum

## Objetivo
Implementación de la estrategia Momentum siguiendo la metodología MSCI adaptada. Este notebook calcula las señales de inversión mensuales y genera el archivo CSV con los 20 activos seleccionados en cada fecha de rebalanceo.

## Metodología (según PDF)
1. **Paso A:** Cálculo de Momentum 12 meses (R_12) y 6 meses (R_6) con lag de 1 mes
2. **Paso B:** Normalización por Z-Score dentro del universo del mes
3. **Paso C:** Score final = media de Z_6 y Z_12, selección top 20, pesos 5% cada uno

## Índice
1. [Configuración y Carga de Datos](#configuracion)
2. [Paso A: Cálculo de Momentum con Lag](#paso-a)
3. [Paso B: Normalización Z-Score](#paso-b)
4. [Paso C: Selección Top 20 y Pesos](#paso-c)
5. [Generación del CSV de Selección](#csv)

---

## 1. Configuración y Carga de Datos {#configuracion}

Carga de datos preparados en notebooks anteriores.

In [ ]:
# Librerías permitidas
import numpy as np
import pandas as pd
import warnings
import os
from datetime import datetime

warnings.filterwarnings('ignore')

# Cargar datos de notebooks anteriores
data_dir = '../data'

# Cargar datos (descomentar cuando estén disponibles)
# log_returns_monthly = pd.read_parquet(f'{data_dir}/log_returns_monthly.parquet')
# rebalance_calendar = pd.read_csv(f'{data_dir}/rebalance_calendar.csv', index_col=0, parse_dates=True)
# prices_monthly = pd.read_parquet(f'{data_dir}/prices_monthly.parquet')
# universe_monthly = pd.read_csv(f'{data_dir}/universe_monthly.csv', index_col=0, parse_dates=True)

print("⚠️  Cargar datos de notebooks anteriores antes de continuar")
print(f"Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Paso A: Cálculo de Momentum con Lag {#paso-a}

**IMPORTANTE:** Según el PDF:
- **Momentum 12 meses (R_12):** Rentabilidad desde mes t-13 al mes t-1 (excluye mes actual)
- **Momentum 6 meses (R_6):** Rentabilidad desde mes t-7 al mes t-1 (excluye mes actual)
- Se utilizan **retornos logarítmicos** para el cálculo

El lag de 1 mes evita el ruido de reversión a la media.

In [ ]:
def calculate_momentum(log_returns_monthly, period=12, lag=1):
    """
    Calcula momentum con lag.
    
    Parámetros:
    -----------
    log_returns_monthly : pd.DataFrame
        Retornos logarítmicos mensuales
    period : int
        Período de momentum (12 o 6 meses)
    lag : int
        Lag en meses (1 = excluye mes actual)
    
    Retorna:
    --------
    pd.DataFrame
        Momentum calculado para cada activo y mes
    """
    # Momentum = suma de retornos logarítmicos en el período
    # Con lag: desde t-period-lag hasta t-lag-1
    momentum = log_returns_monthly.rolling(window=period).sum().shift(lag)
    
    return momentum


def calculate_momentum_12_6(log_returns_monthly, lag=1):
    """
    Calcula Momentum 12 meses y 6 meses con lag.
    
    Parámetros:
    -----------
    log_returns_monthly : pd.DataFrame
        Retornos logarítmicos mensuales
    lag : int
        Lag en meses (1 = excluye mes actual)
    
    Retorna:
    --------
    tuple
        (momentum_12, momentum_6)
    """
    # Momentum 12 meses: desde t-13 hasta t-1 (12 meses de datos)
    momentum_12 = calculate_momentum(log_returns_monthly, period=12, lag=lag)
    
    # Momentum 6 meses: desde t-7 hasta t-1 (6 meses de datos)
    momentum_6 = calculate_momentum(log_returns_monthly, period=6, lag=lag)
    
    return momentum_12, momentum_6


# Calcular momentum (descomentar cuando log_returns_monthly esté disponible)
# momentum_12, momentum_6 = calculate_momentum_12_6(log_returns_monthly, lag=1)

# Verificación
# print("\\n=== VERIFICACIÓN DE MOMENTUM ===")
# print(f"Período de datos: {log_returns_monthly.index.min()} a {log_returns_monthly.index.max()}")
# print(f"\\nMomentum 12 meses - Primeras filas:")
# print(momentum_12.head(15))  # Mostrar más filas para ver el lag
# print(f"\\nMomentum 6 meses - Primeras filas:")
# print(momentum_6.head(10))

print("⚠️  Calcular momentum después de cargar retornos logarítmicos mensuales")

## 3. Paso B: Normalización Z-Score {#paso-b}

Debido a que las volatilidades de los períodos de 6 y 12 meses son distintas, se normalizan los retornos dentro del universo de cada mes usando Z-Score:

$$Z = \\frac{X - \\mu}{\\sigma}$$

Donde:
- $\\mu$ = media de retornos de los activos en ese mes
- $\\sigma$ = desviación estándar de retornos de los activos en ese mes

In [ ]:
def calculate_zscore(momentum_data, universe_monthly, rebalance_dates):
    """
    Calcula Z-Score de momentum normalizado dentro del universo de cada mes.
    
    Parámetros:
    -----------
    momentum_data : pd.DataFrame
        Momentum calculado (R_12 o R_6)
    universe_monthly : pd.DataFrame
        Universo de activos elegibles por mes
    rebalance_dates : pd.DatetimeIndex
        Fechas de rebalanceo
    
    Retorna:
    --------
    pd.DataFrame
        Z-Scores normalizados por mes
    """
    zscore_data = pd.DataFrame(index=momentum_data.index, columns=momentum_data.columns)
    
    for rebalance_date in rebalance_dates:
        if rebalance_date not in momentum_data.index:
            continue
        
        # Obtener universo de activos para este mes
        # (Ajustar según estructura de universe_monthly)
        if isinstance(universe_monthly, pd.DataFrame):
            # Si universe_monthly es DataFrame con lista de activos por fila
            eligible_assets = universe_monthly.loc[rebalance_date].dropna().tolist()
        else:
            # Si es un diccionario o lista simple
            eligible_assets = universe_monthly.get(rebalance_date, momentum_data.columns.tolist())
        
        # Filtrar solo activos elegibles
        eligible_assets = [a for a in eligible_assets if a in momentum_data.columns]
        
        if len(eligible_assets) == 0:
            continue
        
        # Obtener momentum de activos elegibles para este mes
        momentum_values = momentum_data.loc[rebalance_date, eligible_assets].dropna()
        
        if len(momentum_values) < 2:  # Necesitamos al menos 2 valores para calcular std
            continue
        
        # Calcular media y desviación estándar del universo
        mu = momentum_values.mean()
        sigma = momentum_values.std()
        
        if sigma == 0:  # Evitar división por cero
            zscore_data.loc[rebalance_date, eligible_assets] = 0
        else:
            # Calcular Z-Score: (X - mu) / sigma
            zscore = (momentum_values - mu) / sigma
            zscore_data.loc[rebalance_date, eligible_assets] = zscore
    
    return zscore_data


# Calcular Z-Scores (descomentar cuando momentum y universe estén disponibles)
# zscore_12 = calculate_zscore(momentum_12, universe_monthly, rebalance_calendar.index)
# zscore_6 = calculate_zscore(momentum_6, universe_monthly, rebalance_calendar.index)

# Verificación
# print("\\n=== VERIFICACIÓN DE Z-SCORES ===")
# print(f"\\nZ-Score 12 meses - Estadísticas:")
# print(zscore_12.describe().T.head(10))
# print(f"\\nZ-Score 6 meses - Estadísticas:")
# print(zscore_6.describe().T.head(10))

print("⚠️  Calcular Z-Scores después de calcular momentum")

In [ ]:
def calculate_final_score_and_select(zscore_6, zscore_12, top_n=20, weight_per_asset=0.05):
    """
    Calcula score final y selecciona top N activos con pesos iguales.
    
    Parámetros:
    -----------
    zscore_6 : pd.DataFrame
        Z-Scores de momentum 6 meses
    zscore_12 : pd.DataFrame
        Z-Scores de momentum 12 meses
    top_n : int
        Número de activos a seleccionar (default: 20)
    weight_per_asset : float
        Peso por activo (default: 0.05 = 5%)
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame con activos seleccionados y pesos por fecha de rebalanceo
    """
    # Score final = media de Z_6 y Z_12
    final_score = (zscore_6 + zscore_12) / 2
    
    # DataFrame para almacenar selección y pesos
    selection_data = []
    
    for date in final_score.index:
        # Obtener scores válidos para esta fecha
        scores = final_score.loc[date].dropna().sort_values(ascending=False)
        
        if len(scores) == 0:
            continue
        
        # Seleccionar top N
        top_assets = scores.head(top_n).index.tolist()
        
        # Crear registro
        for i, asset in enumerate(top_assets):
            selection_data.append({
                'rebalance_date': date,
                'asset': asset,
                'rank': i + 1,
                'score': scores[asset],
                'weight': weight_per_asset
            })
    
    selection_df = pd.DataFrame(selection_data)
    
    return selection_df, final_score


# Calcular selección (descomentar cuando zscores estén disponibles)
# selection_df, final_scores = calculate_final_score_and_select(
#     zscore_6, 
#     zscore_12, 
#     top_n=20, 
#     weight_per_asset=0.05
# )

# Verificación
# print("\\n=== VERIFICACIÓN DE SELECCIÓN ===")
# print(f"Total de selecciones: {len(selection_df)}")
# print(f"Fechas de rebalanceo únicas: {selection_df['rebalance_date'].nunique()}")
# print(f"\\nPrimera fecha de rebalanceo:")
# first_date = selection_df['rebalance_date'].min()
# print(selection_df[selection_df['rebalance_date'] == first_date][['asset', 'rank', 'score', 'weight']])

print("⚠️  Calcular selección después de calcular Z-Scores")

## 5. Generación del CSV de Selección {#csv}

Según el PDF, se debe generar un archivo CSV con los 20 activos seleccionados en cada fecha de rebalanceo.

In [ ]:
def generate_selection_csv(selection_df, output_path='../data/selected_assets.csv'):
    """
    Genera archivo CSV con activos seleccionados por fecha de rebalanceo.
    
    Parámetros:
    -----------
    selection_df : pd.DataFrame
        DataFrame con selección de activos
    output_path : str
        Ruta de salida para el CSV
    """
    # Formatear para CSV: una fila por fecha con los 20 activos
    csv_data = []
    
    for date in selection_df['rebalance_date'].unique():
        date_selection = selection_df[selection_df['rebalance_date'] == date].sort_values('rank')
        assets = date_selection['asset'].tolist()
        
        # Asegurar que hay exactamente 20 activos (rellenar con NaN si faltan)
        while len(assets) < 20:
            assets.append('')
        
        row = {'rebalance_date': date}
        for i in range(20):
            row[f'asset_{i+1}'] = assets[i] if i < len(assets) else ''
        
        csv_data.append(row)
    
    csv_df = pd.DataFrame(csv_data)
    csv_df.set_index('rebalance_date', inplace=True)
    
    # Guardar CSV
    csv_df.to_csv(output_path)
    print(f"\\n✓ CSV guardado en: {output_path}")
    print(f"  Total de fechas: {len(csv_df)}")
    print(f"  Columnas: {list(csv_df.columns)}")
    
    return csv_df


# Generar CSV (descomentar cuando selection_df esté disponible)
# selection_csv = generate_selection_csv(selection_df, output_path='../data/selected_assets.csv')

# Guardar también datos intermedios para uso en siguientes notebooks
# selection_df.to_csv('../data/selection_with_weights.csv')
# final_scores.to_parquet('../data/final_scores.parquet')

print("\\n=== RESUMEN DEL NOTEBOOK 3 ===")
print("✓ Cálculo de Momentum 12 y 6 meses con lag")
print("✓ Normalización Z-Score por universo mensual")
print("✓ Selección Top 20 con pesos 5%")
print("✓ Generación de CSV de selección")
print("\\n⚠️  IMPORTANTE: Ejecutar todas las celdas con datos reales")
print("⚠️  El CSV generado será usado en el Notebook 4 para el backtesting")